In [71]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [72]:
mc_url = "https://www.mcdonalds.com/ua/uk-ua/eat/fullmenu.html"
response = requests.get(mc_url)

In [73]:
response.apparent_encoding

'utf-8'

In [75]:
soup = BeautifulSoup(response.text)

In [76]:
# parcing index page to get a list of links to detailed description
soup_list = soup.find_all('li', {'class': 'cmp-category__item'})
links = {}
for soup1 in soup_list:
    name = soup1.find('div', {'class': 'cmp-category__item-name'}).get_text()
    link = soup1.find('a').get('href')
    
    links[name] = link
links

{'Роял Делюкс': '/ua/uk-ua/product/200390.html',
 'Біг Мак®': '/ua/uk-ua/product/200148.html',
 'Гамбургер': '/ua/uk-ua/product/200154.html',
 'Чізбургер': '/ua/uk-ua/product/200150.html',
 'Дабл Чізбургер': '/ua/uk-ua/product/200152.html',
 'МакЧікен': '/ua/uk-ua/product/200155.html',
 'Біг Тейсті®': '/ua/uk-ua/product/200149.html',
 'Філе-о-фіш®': '/ua/uk-ua/product/200153.html',
 'Роял Чізбургер': '/ua/uk-ua/product/200257.html',
 'Дабл Роял Чізбургер': '/ua/uk-ua/product/200266.html',
 'Чізбургер з беконом': '/ua/uk-ua/product/200151.html',
 'Чікен Джуніор': '/ua/uk-ua/product/200372.html',
 'МакТост® із сиром': '/ua/uk-ua/product/200010.html',
 'Чікен Рол': '/ua/uk-ua/product/200136.html',
 'Фіш Рол': '/ua/uk-ua/product/200137.html',
 'Чікен МакНагетс® 4шт': '/ua/uk-ua/product/200026.html',
 'Чікен МакНагетс 6шт': '/ua/uk-ua/product/200027.html',
 'Чікен МакНагетс 9шт': '/ua/uk-ua/product/200028.html',
 'Чікен МакНагетс 20 шт': '/ua/uk-ua/product/2133.html',
 'Курячі Стріпси, 3 шт

In [80]:
### parcing was not paralleled, so it takes some time ###
# to look at all parced data - see parce_results.txt

# parcing pages with data to gather nessesary data.
# selenium used for clicking buttons that expands details

driver = webdriver.Chrome('chromedriver.exe')
base_url = 'https://www.mcdonalds.com'
all_vals = []
i=0
for link in links.values():
    full_link = base_url + link
    print(full_link)
    # time.sleep(10)
    try:
        # clicking button
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(full_link)
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "cmp-accordion__button"))).click()
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        driver.close()
        driver.quit()
        
        # getting first part of useful data in header
        name = soup.find('span', {'class':'cmp-product-details-main__heading-title'}).get_text()
        description = soup.find('div', {'class':'cmp-product-details-main'}).find('div', {'class':'cmp-text'}).get_text()
        vals = (soup.find('div', {'class':'cmp-nutrition-summary cmp-nutrition-summary--primary'})
                .find_all('li', {'class': "cmp-nutrition-summary__heading-primary-item"}))
        for ind, mini_soup in enumerate(vals):
            if ind == 0:
                calories = mini_soup.find('span', {'class': 'sr-only sr-only-pd'}).get_text().strip()
                # calories2 = mini_soup.find('span', {'class': 'metric'}).find('span', {'class': 'sr-only sr-only-pd'}).get_text()
            if ind == 1:
                fats = mini_soup.find('span', {'class': 'sr-only sr-only-pd'}).get_text().strip()
            if ind == 2:
                carbs = mini_soup.find('span', {'class': 'sr-only sr-only-pd'}).get_text().strip()
            if ind == 3:
                proteins = mini_soup.find('span', {'class': 'sr-only sr-only-pd'}).get_text().strip()
        
        
        # reading fats, sugar, salt, portion from a section hidden
        vals2 = (soup.find('div', {'class':'cmp-nutrition-summary__details-column-view-desktop'})
                .find_all('li', {'class': 'label-item'}))
        
        for ind, vals in enumerate(vals2):
            if ind == 0:
                unsaturated_fats = vals.find('span', {'class': 'sr-only'}).get_text().strip()
                # calories2 = mini_soup.find('span', {'class': 'metric'}).find('span', {'class': 'sr-only sr-only-pd'}).get_text()
            if ind == 1:
                sugar = vals.find('span', {'class': 'sr-only'}).get_text().strip()
            if ind == 2:
                salt = vals.find('span', {'class': 'sr-only'}).get_text().strip()
            if ind == 3:
                portion = vals.find('span', {'class': 'sr-only'}).get_text().strip()
        
        
        print(name, description, calories, fats, carbs, proteins, unsaturated_fats, sugar, salt, portion)
        import re
        values = {
            "name":re.sub(' +', ' ',name),
            "description":re.sub(' +', ' ',description),
            "calories":re.sub(' +', ' ',calories),
            "fats":re.sub(' +', ' ',fats),
            "carbs":re.sub(' +', ' ',carbs),
            "proteins":re.sub(' +', ' ',proteins),
            "unsaturated_fats":re.sub(' +', ' ',unsaturated_fats),
            "sugar":re.sub(' +', ' ',sugar),
            "salt":re.sub(' +', ' ',salt),
            "portion":re.sub(' +', ' ',portion),
        }
    except Exception as e:
        print(e.args)
        values = {'error': e.args}
    all_vals.append(values)
    # code for testing
    # i += 1
    # if i == 2:
    #     break

import json
with open('parce_results.txt', 'w') as fd:
    fd.write(json.dumps(all_vals)) 




https://www.mcdonalds.com/ua/uk-ua/product/200390.html
Роял Делюкс 
Яловича котлетка, огорнута сиром чедер, а також свіжий салат, томат, мариновані огірочки, цибуля та мікс гірчиці, кетчупа й соуса створюють незабутній смак. Спробуй свіжу новинку — Роял Делюкс.
 560ккал Калорійність
                    (28 % Daily Value) 32.3г Жири
                    (48 % Daily Value) 39.7г Вуглеводи
                    (14 % Daily Value) 28.8г Білки
                    (39 % Daily Value) 12.3г
            (55 Percent Daily Values ) 9.8г
            (9 Percent Daily Values ) 2.6г
            (54 Percent Daily Values ) 220г
            (N/A Percent Daily Values )
            N/Aг per 100 grams
https://www.mcdonalds.com/ua/uk-ua/product/200148.html
Біг Мак® 
						Два біфштекси з натуральної яловичини, цибуля, маринований огірок, сир “Чеддер”, свіжий салат, заправлені спеціальним соусом, у булочці з насінням сезаму.
					 526ккал Калорійність
                    (26 % Daily Value) 27г Жири
            

In [78]:
# preview of data
with open('parce_results.txt', 'r') as fd:
    print(json.load(fd))


[{'name': 'Роял Делюкс', 'description': 'Яловича котлетка, огорнута сиром чедер, а також свіжий салат, томат, мариновані огірочки, цибуля та мікс гірчиці, кетчупа й соуса створюють незабутній смак. Спробуй свіжу новинку — Роял Делюкс.', 'calories': '560ккал Калорійність                    (28 % Daily Value)', 'fats': '32.3г Жири                    (48 % Daily Value)', 'carbs': '39.7г Вуглеводи                    (14 % Daily Value)', 'proteins': '28.8г Білки                    (39 % Daily Value)', 'unsaturated fats': '12.3г            (55 Percent Daily Values )', 'sugar': '9.8г            (9 Percent Daily Values )', 'salt': '2.6г            (54 Percent Daily Values )', 'portion': '220г            (N/A Percent Daily Values )            N/Aг per 100 grams'}, {'name': 'Біг Мак®', 'description': 'Два біфштекси з натуральної яловичини, цибуля, маринований огірок, сир “Чеддер”, свіжий салат, заправлені спеціальним соусом, у булочці з насінням сезаму.', 'calories': '526ккал Калорійність       